In [1]:
%matplotlib inline
from env import user, password, host
from math import sqrt 
from matplotlib.colors import ListedColormap
from scipy import stats
from scipy.stats import spearmanr, pearsonr
from sklearn import neighbors, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer, PolynomialFeatures
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import statsmodels.api as sm
import warnings


plt.rc('figure', figsize=(13, 7))
plt.rc('font', size=16)
plt.style.use('seaborn-whitegrid')
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('DISTRICT_summary_22.csv')

In [3]:
df.head()

,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,CHARTER_STATUS,SECTION,HEADING,HEADING NAME,YR22
0,DISTRICT SUMMARY,10.0,A W BROWN LEADERSHIP ACADEMY,57816.0,OPEN ENROLLMENT CHARTER,A-PARTICIPATION,A01,DISTRICT CUMULATIVE YEAR END ENROLLMENT,1177.0
1,DISTRICT SUMMARY,10.0,A W BROWN LEADERSHIP ACADEMY,57816.0,OPEN ENROLLMENT CHARTER,A-PARTICIPATION,A02,DISTRICT DISCIPLINE POPULATION,156.0
2,DISTRICT SUMMARY,10.0,A W BROWN LEADERSHIP ACADEMY,57816.0,OPEN ENROLLMENT CHARTER,A-PARTICIPATION,A03,DISTRICT DISCIPLINE RECORD COUNT,248.0
3,DISTRICT SUMMARY,10.0,A W BROWN LEADERSHIP ACADEMY,57816.0,OPEN ENROLLMENT CHARTER,B-DISCIPLINE DATA TRENDS,B10,COUNT OF STUDENTS SUSPENDED IN SCHOOL,45.0
4,DISTRICT SUMMARY,10.0,A W BROWN LEADERSHIP ACADEMY,57816.0,OPEN ENROLLMENT CHARTER,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,132.0


In [4]:
df.tail()

,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,CHARTER_STATUS,SECTION,HEADING,HEADING NAME,YR22
66486,DISTRICT SUMMARY,15.0,ZEPHYR ISD,25906.0,TRADITIONAL ISD/CSD,W-REASON INCIDENT COUNTS,G61,61-BULLYING  TEC 37.0052(B),-999.0
66487,DISTRICT SUMMARY,15.0,ZEPHYR ISD,25906.0,TRADITIONAL ISD/CSD,X-DISCIPLINE ACTION COUNTS,H05,05-OUT-OF-SCHOOL SUSPENSION,-999.0
66488,DISTRICT SUMMARY,15.0,ZEPHYR ISD,25906.0,TRADITIONAL ISD/CSD,X-DISCIPLINE ACTION COUNTS,H06,06-IN-SCHOOL SUSPENSION,-999.0
66489,-999 indicates counts or percentages are not a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66490,Masked numbers are typically small although la...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df.rename(columns={'AGGREGATION LEVEL':'agg_level', 'REGION':'region', 'DISTNAME':'dist_name', 'DISTRICT':'district_num', 'CHARTER_STATUS':'charter_status', 'SECTION':'section', 'HEADING':'heading', 'HEADING NAME':'heading_name', 'YR22': 'num_of_students'})

In [6]:
df.shape

(66491, 9)

In [7]:
df.columns

Index(['agg_level', 'region', 'dist_name', 'district_num', 'charter_status',
       'section', 'heading', 'heading_name', 'num_of_students'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66491 entries, 0 to 66490
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   agg_level        66491 non-null  object 
 1   region           66489 non-null  float64
 2   dist_name        66489 non-null  object 
 3   district_num     66489 non-null  float64
 4   charter_status   66489 non-null  object 
 5   section          66489 non-null  object 
 6   heading          66489 non-null  object 
 7   heading_name     66489 non-null  object 
 8   num_of_students  66489 non-null  float64
dtypes: float64(3), object(6)
memory usage: 4.6+ MB


In [9]:
 #return a new dataframe by dropping rows
# from dataframe using their
# respective index position
df = df.drop([df.index[66489], df.index[66490]])

In [10]:
df['charter_encoded'] = df.charter_status.map({'OPEN ENROLLMENT CHARTER': 1, 'TRADITIONAL ISD/CSD':0})

In [11]:
df=df[(df.heading == 'A01') | (df.heading ==  'A03')]

In [12]:
df['heading_encoded'] = df.heading.map({'A01': 1, 'A03':0})

In [13]:
df[(df.num_of_students != -999)]

,agg_level,region,dist_name,district_num,charter_status,section,heading,heading_name,num_of_students,charter_encoded,heading_encoded
0,DISTRICT SUMMARY,10.0,A W BROWN LEADERSHIP ACADEMY,57816.0,OPEN ENROLLMENT CHARTER,A-PARTICIPATION,A01,DISTRICT CUMULATIVE YEAR END ENROLLMENT,1177.0,1,1
2,DISTRICT SUMMARY,10.0,A W BROWN LEADERSHIP ACADEMY,57816.0,OPEN ENROLLMENT CHARTER,A-PARTICIPATION,A03,DISTRICT DISCIPLINE RECORD COUNT,248.0,1,0
33,DISTRICT SUMMARY,10.0,A+ ACADEMY,57829.0,OPEN ENROLLMENT CHARTER,A-PARTICIPATION,A01,DISTRICT CUMULATIVE YEAR END ENROLLMENT,1620.0,1,1
35,DISTRICT SUMMARY,10.0,A+ ACADEMY,57829.0,OPEN ENROLLMENT CHARTER,A-PARTICIPATION,A03,DISTRICT DISCIPLINE RECORD COUNT,428.0,1,0
92,DISTRICT SUMMARY,4.0,A+ UNLIMITED POTENTIAL,101871.0,OPEN ENROLLMENT CHARTER,A-PARTICIPATION,A01,DISTRICT CUMULATIVE YEAR END ENROLLMENT,171.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
66359,DISTRICT SUMMARY,1.0,ZAPATA COUNTY ISD,253901.0,TRADITIONAL ISD/CSD,A-PARTICIPATION,A03,DISTRICT DISCIPLINE RECORD COUNT,550.0,0,0
66425,DISTRICT SUMMARY,7.0,ZAVALLA ISD,3906.0,TRADITIONAL ISD/CSD,A-PARTICIPATION,A01,DISTRICT CUMULATIVE YEAR END ENROLLMENT,338.0,0,1
66427,DISTRICT SUMMARY,7.0,ZAVALLA ISD,3906.0,TRADITIONAL ISD/CSD,A-PARTICIPATION,A03,DISTRICT DISCIPLINE RECORD COUNT,74.0,0,0
66461,DISTRICT SUMMARY,15.0,ZEPHYR ISD,25906.0,TRADITIONAL ISD/CSD,A-PARTICIPATION,A01,DISTRICT CUMULATIVE YEAR END ENROLLMENT,224.0,0,1


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2334 entries, 0 to 66463
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   agg_level        2334 non-null   object 
 1   region           2334 non-null   float64
 2   dist_name        2334 non-null   object 
 3   district_num     2334 non-null   float64
 4   charter_status   2334 non-null   object 
 5   section          2334 non-null   object 
 6   heading          2334 non-null   object 
 7   heading_name     2334 non-null   object 
 8   num_of_students  2334 non-null   float64
 9   charter_encoded  2334 non-null   int64  
 10  heading_encoded  2334 non-null   int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 218.8+ KB


In [15]:
df.tail()

,agg_level,region,dist_name,district_num,charter_status,section,heading,heading_name,num_of_students,charter_encoded,heading_encoded
66359,DISTRICT SUMMARY,1.0,ZAPATA COUNTY ISD,253901.0,TRADITIONAL ISD/CSD,A-PARTICIPATION,A03,DISTRICT DISCIPLINE RECORD COUNT,550.0,0,0
66425,DISTRICT SUMMARY,7.0,ZAVALLA ISD,3906.0,TRADITIONAL ISD/CSD,A-PARTICIPATION,A01,DISTRICT CUMULATIVE YEAR END ENROLLMENT,338.0,0,1
66427,DISTRICT SUMMARY,7.0,ZAVALLA ISD,3906.0,TRADITIONAL ISD/CSD,A-PARTICIPATION,A03,DISTRICT DISCIPLINE RECORD COUNT,74.0,0,0
66461,DISTRICT SUMMARY,15.0,ZEPHYR ISD,25906.0,TRADITIONAL ISD/CSD,A-PARTICIPATION,A01,DISTRICT CUMULATIVE YEAR END ENROLLMENT,224.0,0,1
66463,DISTRICT SUMMARY,15.0,ZEPHYR ISD,25906.0,TRADITIONAL ISD/CSD,A-PARTICIPATION,A03,DISTRICT DISCIPLINE RECORD COUNT,10.0,0,0


In [ ]:
# Creating a list of our numeric columns
numcols = [col for col in df.columns if df[col].dtype != 'O']

In [ ]:
# Creating a list of our categorical columns
catcols = [col for col in df.columns if df[col].dtype == 'O']

In [ ]:
# Describe the object columns
for col in catcols:
    print(f"Column: {col}")
    print(df[col].value_counts())
    print("--------")
    print(df[col].value_counts(normalize=True, dropna=False))
    print("=================")

In [ ]:
# Histograms of numeric columns
for col in numcols:
    print(col)
    df[col].hist()
    plt.show()

In [ ]:
df = df.drop_duplicates()

In [ ]:
sns.boxplot(data=train, x='species', y='petal_width')
plt.show

In [16]:
df.shape

(2334, 11)

In [24]:
df.pivot_table(index='district_num', columns='heading', values='heading_name')

heading
district_num
1902.0
1903.0
1904.0
1906.0
1907.0
...
252902.0
252903.0
253901.0


In [17]:
df.describe()

,region,district_num,num_of_students,charter_encoded,heading_encoded
count,2334.000000,2334.000000,2334.000000,2334.000000,2334.000000
mean,10.472579,122764.976007,3100.125536,0.136247,0.517138
std,5.336983,72846.905427,10208.688544,0.343124,0.499813
min,1.000000,1902.000000,-999.000000,0.000000,0.000000
25%,6.000000,60914.000000,153.000000,0.000000,0.000000
50%,10.000000,110907.500000,540.000000,0.000000,1.000000
75%,15.000000,182905.750000,1881.750000,0.000000,1.000000
max,20.000000,254902.000000,211658.000000,1.000000,1.000000


In [18]:
df[df[['district_num', 'heading_name', 'num_of_students']].duplicated()]

,agg_level,region,dist_name,district_num,charter_status,section,heading,heading_name,num_of_students,charter_encoded,heading_encoded


In [20]:
df1=df.pivot(index='district_num', columns='heading_name', values= 'num_of_students')

In [21]:
df1.head()

heading_name,DISTRICT CUMULATIVE YEAR END ENROLLMENT,DISTRICT DISCIPLINE RECORD COUNT
district_num,,
1902.0,609.0,62.0
1903.0,1301.0,246.0
1904.0,868.0,204.0
1906.0,367.0,69.0
1907.0,3648.0,1031.0


In [ ]:
df1

In [ ]:
pd.crosstab(df.embarked, df.embark_town)

In [ ]:
def split_tea_data(df):
    '''
    This function performs split on telco data, stratify churn.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.charter_encoded)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.charter_encoded)
    return train, validate, test

In [ ]:
train, validate, test= split_tea_data(df) 

In [ ]:
train.shape

In [ ]:
validate.shape

In [ ]:
test.shape

In [ ]:
for col in train.columns:
    plt.hist(train[col])
    plt.title(col)
    plt.show()


In [ ]:
sns.pairplot(train, hue='charter_encoded', corner=True)
plt.show()


In [ ]:
train.corr()

In [ ]:
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(train.corr(method='spearman')[['home_value']].sort_values(by='home_value', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with Home Price', fontdict={'fontsize':18}, pad=16);

In [ ]:
sns.heatmap(train.corr(), cmap='mako', center=0, annot=True)
plt.show()

In [ ]:
train.head()

In [ ]:
def scale_data(train, 
               validate, 
               test, 
               columns_to_scale = ['region', 'district_num','charter_encoded', 'heading_encoded',  'num_of_students'],
               return_scaler=False):
    '''
    Scales the 3 data splits. 
    Takes in train, validate, and test data splits and returns their scaled counterparts.
    If return_scalar is True, the scaler object will be returned as well
    '''
    # make copies of our original data so we dont gronk up anything
    train_scaled = train.copy()
    validate_scaled = validate.copy()
    test_scaled = test.copy()
    #     make the thing
    scaler = MinMaxScaler()
    #     fit the thing
    scaler.fit(train[columns_to_scale])
    # applying the scaler:
    train_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(train[columns_to_scale]),
                                                  columns=train[columns_to_scale].columns.values).set_index([train.index.values])
                                                  
    validate_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(validate[columns_to_scale]),
                                           columns=validate[columns_to_scale].columns.values).set_index([validate.index.values])
    
    test_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(test[columns_to_scale]),
                                                 columns=test[columns_to_scale].columns.values).set_index([test.index.values])
    
    if return_scaler:
        return scaler, train_scaled, validate_scaled, test_scaled
    else:
        return train_scaled, validate_scaled, test_scaled

In [ ]:
train_scaled, validate_scaled, test_scaled=scale_data(train, validate, test)

In [ ]:
train_scaled.head()

In [ ]:
validate_scaled.head()

In [ ]:
test_scaled.head()

In [ ]:
spearman_train_corr= train.corr(method='spearman')
spearman_train_corr

In [ ]:
train_scaled.num_of_students.hist()


In [ ]:
df.charter_encoded.value_counts()

In [ ]:
train_scaled.groupby('heading', group_keys=True).apply(lambda x: x)
      

In [ ]:
df.groupby("Animal", group_keys=False).apply(lambda x: x)

In [ ]:

tstat, p = stats.ttest_ind(df.sepal_length,
                versicolor.sepal_length, 
                equal_var=False
               )

In [ ]:

tstat

In [ ]:
p

In [ ]:
stats.mannwhitneyu(virginica.sepal_width, 
                   versicolor.sepal_width
                  )

In [ ]:
train['age_bin'] = pd.cut(train.age, [0, 40, 80])
train


In [ ]:
df = df.rename(columns={'sale_amount': 'quantity'})

In [ ]:
df = df.fillna(0)

In [ ]:
#Distribution of target variable
plt.hist(x=df['logerror'], bins=20)
plt.xlim(-1, 1)


In [ ]:
#Visualize distribution of target variable
plt.hist(df['spending_score'])


In [ ]:
df.columns

In [ ]:
 # Get dummies for non-binary categorical variables
    dummy1_df = pd.get_dummies(df[['section', \
                              'heading', \
                              'heading_name', \
                              'device_protection', \
                              'streaming_tv', \
                              'streaming_movies', \
                              'internet_service_type'
                              ]], dummy_na=False, \
                              drop_first=True)
    

In [ ]:
#Split my data into three subsets
seed = 123

train, test_val = train_test_split(mall_df, train_size=0.7,
                                   random_state=seed)

test, val = train_test_split(test_val, train_size=0.5,
                                       random_state=seed)

mall_train.shape, mall_val.shape, mall_test.shape


In [ ]:
# 80/20 train test split
train, test = train_test_split(df, train_size=0.8, random_state=1349, stratify=df.species)
# 70/30 train validate split
train, validate = train_test_split(train, train_size=0.7, random_state=1349, stratify=train.species)


In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
#Check get_dummies works as expected on my training data
pd.get_dummies(mall_train)

In [ ]:
#Create the dataframe with dummies for gender and drop redundant column
mall_train = pd.get_dummies(mall_train)

mall_train.drop(columns=['gender_Male'], inplace=True)

mall_train.head()


In [ ]:
#Initialize the scaler and fit/transform a couple columns in my training data
mms = MinMaxScaler()

mall_train[['age', 'annual_income']] = mms.fit_transform(mall_train[['age',
                                                                     'annual_income']])

mall_train.head()


In [ ]:
stats.levene(
    train[train.age <= 40].spending_score,
    train[train.age > 40].spending_score
)

In [ ]:
stats.ttest_ind(
    train[train.age <= 40].spending_score,
    train[train.age > 40].spending_score,
    equal_var=False
)

In [ ]:
Exercises Part 1 Continue in your classification_exercises.ipynb notebook. As always, add, commit, and push your changes. Section 1 - iris_db: Using iris data from our mySQL server and the methods used in the lesson above: Acquire, prepare & split your data.

Target: Species

Null H:There is no difference in Sepal length between species.

Alt H: There is a difference in Sepal length between species.

Univariate Stats For each measurement type (quantitative variable): create a histogram, boxplot, & compute descriptive statistics (using .describe()). For each species (categorical variable): create a frequency table and a bar plot of those frequencies. Document takeaways & any actions.



In [ ]:
Takeaway: Categorical column (species are even distributed) due to training split with 28 of each species.

Bivariate Stats

Visualize each measurement type (y-axis) with the species variable (x-axis) using barplots, adding a horizontal line showing the overall mean of the metric (y-axis).

For each measurement type, compute the descriptive statistics for each species.

For virginica & versicolor: Compare the mean petal_width using the Mann-Whitney test (scipy.stats.mannwhitneyu) to see if there is a significant difference between the two groups. Do the same for the other measurement types.

Document takeaways & any actions.

In [ ]:
Multivariate Stats

Visualize the interaction of each measurement type with the others using a pairplot (or scatter matrix or something similar) and add color to represent species.

Visualize two numeric variables of the species. Hint: sns.relplot with hue or col

Create a swarmplot using a melted dataframe of all your numeric variables. The x-axis should be the variable name, the y-axis the measure. Add another dimension using color to represent species. Document takeaways from this visualization.

Ask a specific question of the data, such as: is the sepal area signficantly different in virginica compared to setosa? Answer the question through both a plot and using a mann-whitney or t-test. If you use a t-test, be sure assumptions are met (independence, normality, equal variance).

Document takeaways and any actions.